# Statistics

This notebook goes through a release folder of MGnify. Fistrly it analyzes the release folder structure to get the number of clusters discarded due to BIAS, due to UniProt matches or to MGnify matches and other useful statistics.

Afterwards, it checks overall distributions of SEED multiple sequence alignments statistics, such as dimensions, occupancy score, conservation score and prettiness score. Moreover, it compares pre-trimming and post-trimming SEED multiple sequence alignments and retrieves.

In [ ]:
!pwd

In [ ]:
# Dependencies
import matplotlib.pyplot as plt
from glob import glob, iglob
from os import path
import numpy as np
import random
import sys
import os

# Define random seed
random.seed(42)

# Setup plotting
%matplotlib inline

In [ ]:
# Add custom dependecnies to path
sys.path.append(os.path.join(os.getcwd(), '..'))

# Custom deipendencies
from src.msa import MSA
from src.msa import consensus, occupancy, conservation, prettiness

In [ ]:
# Constants
CLUSTERS_PATH = os.path.join('..', 'tmp', 'mgseed2', 'MGYP*')

In [ ]:
# # Define input file path
# in_path = '/hps/nobackup2/production/metagenomics/dclementel/MGnifam_build/Batch_lists/no_pfam_no_uniprot.txt'
# 
# # Define output file path 
# out_path = os.path.join(os.getcwd(), '..', 'tmp', 'sample')
# 
# # Take sample of size less than total number of clusters
# sample = set(random.sample(range(1, int(1e06)), int(1e03)))
# 
# # Open input file
# in_file = open(in_path, 'r')
# # Open output file
# out_file = open(out_path, 'w')
# # Get clusters file
# for i, line in enumerate(in_file):
#     # Case index is in sample
#     if i in sample:
#         # Write line to output file
#         out_file.write(line)
#         # Remove from sample (early stopping)
#         sample.remove(i)
#     # Stoping condition: sample is empty
#     if len(sample) == 0:
#         break
# # Close input file
# in_file.close()
# # Close output file
# out_file.close()

## Batch and build statistics

In [ ]:
# # Get number of passed clusters, discarded due to bias, to UniProt or to MGnify
# def get_stats(batch_path):
#     
#     # Define iterator through passed clusters
#     cluster_iter = iglob(path.join(batch_path, 'MGYP*'))
#     # Define iterator through BIAS clusters
#     bias_iter = iglob(path.join(batch_path, 'BIAS', 'MGYP*'))
#     # Define iterator through UniProt folder
#     uniprot_iter = iglob(path.join(batch_path, 'UniProt', 'MGYP*'))
#     # Define iterator through MGnify folder
#     mgnify_iter = iglob(path.join(batch_path, 'MGnify', 'MGYP*'))
#     
#     # Define number of passed clusters
#     num_passed = sum([1 for i in cluster_iter])
#     # Define number of BIAS clusters
#     num_bias = sum([1 for i in bias_iter])
#     # Define number of UniProt clusters
#     num_uniprot = sum([1 for i in uniprot_iter])
#     # Define number of MGnify clusters
#     num_mgnify = sum([1 for i in mgnify_iter])
#     
#     # Return statistics
#     return num_passed, num_bias, num_uniprot, num_mgnify

## SEED statistics

In [ ]:
# Get occupancy value and conservation bit-score
def get_msa_stats(msa_path):
    
    # Load MSA from file
    msa = MSA.from_aln(msa_path)
    # Get MSA shape
    n, m = msa.aln.shape
    
    # Get consensus
    cns = consensus(msa.aln)
    # Use consensus to compute occupancy
    occ = occupancy(cns)
    # Use consensus to compute conservation
    csv = conservation(cns)
    # Use conservation to compute prettiness score
    prt = prettiness(csv, n, m)
    
    # Return scores
    return occ, csv, prt, (n, m)

In [ ]:
# Plot statistics
def plot_msa_stats(stats, label_size=18, title_size=24, figsize=(30, 10),
                   shape_xlim=None, shape_ylim=None,
                   occ_xlim=None, occ_ylim=None,
                   csv_xlim=None, csv_ylim=None):
    # Make plot
    fig, axs = plt.subplots(1, 3, figsize=figsize)
    # Set orizontal titles
    axs[0].set_title('Shape', fontsize=title_size)
    axs[1].set_title('Occupancy', fontsize=title_size)
    axs[2].set_title('Conservation', fontsize=title_size)
    # Set labels for SEED shape
    axs[0].set_xlabel('Width', fontsize=label_size)
    axs[0].set_ylabel('Height', fontsize=label_size)
    # Set lims for SEED shape
    axs[0].set_xlim(*shape_xlim if shape_xlim is not None else None)
    axs[0].set_ylim(*shape_ylim if shape_ylim is not None else None)
    # Set labels for SEED occupancy
    axs[1].set_xlabel('Occupancy score', fontsize=label_size)
    axs[1].set_ylabel('Frequencies', fontsize=label_size)
    # Set lims for SEED occupancy
    axs[1].set_xlim(*occ_xlim if occ_xlim is not None else None)
    axs[1].set_ylim(*occ_ylim if occ_ylim is not None else None)
    # Set labels for SEED conservation
    axs[2].set_xlabel('Conservation score', fontsize=label_size)
    axs[2].set_ylabel('Frequencies', fontsize=label_size)
    # Set lims for SEED conservation
    axs[2].set_xlim(*csv_xlim if csv_xlim is not None else None)
    axs[2].set_ylim(*csv_ylim if csv_ylim is not None else None)
    # Plot MSA width vs height# 
    axs[0].scatter(x=stats.get('width'), y=stats.get('height'))  # Make scatterplot
    # Plot MSA occupancy distribution
    axs[1].hist(stats.get('occ'), bins=100, density=False)  # Make histogram
    axs[1].axvline(np.mean(stats.get('occ')), color='r') # Add mean line
    # Plot MSA conservation distribution
    axs[2].hist(stats.get('csv'), bins=100, density=False)  # Make histogram
    axs[2].axvline(np.mean(stats.get('csv')), color='r') # Add mean line
    # Show plot
    plt.show()
    # Close plot
    plt.close()

In [ ]:
# Get MSA statistics

# Initialize statistics dictionary
stats_raw, stats_trim = dict(), dict()

# Define iterator thorugh every passed cluster in every batch
clusters_iter = iglob(CLUSTERS_PATH)

# Loop through every passed cluster
for cluster_path in clusters_iter:
    # Define current cluster's raw SEED alignment path
    seed_raw_path = path.join(cluster_path, 'SEED_raw')
    # Define current cluster's trimmed SEED alignment path
    seed_trim_path = path.join(cluster_path, 'SEED')
    
    # Get statistics from raw SEED alignment
    occ, csv, prt, (n, m) = get_msa_stats(seed_raw_path)
    stats_raw.setdefault('occ', []).extend(occ)
    stats_raw.setdefault('csv', []).extend(csv)
    stats_raw.setdefault('prt', []).append(prt)
    stats_raw.setdefault('width', []).append(m)
    stats_raw.setdefault('height', []).append(n)
    # Get statistics from trimmed SEED alignment
    occ, csv, prt, (n, m) = get_msa_stats(seed_trim_path)
    stats_trim.setdefault('occ', []).extend(occ)
    stats_trim.setdefault('csv', []).extend(csv)
    stats_trim.setdefault('prt', []).append(prt)
    stats_trim.setdefault('width', []).append(m)
    stats_trim.setdefault('height', []).append(n)

In [ ]:
# Plot before-trimming alignment size, occupancy and conservation
plot_msa_stats(stats_raw, figsize=(28, 7),
               shape_xlim=(0, 4000), shape_ylim=(0, 700),
               occ_xlim=(0, 1), occ_ylim=(0, 14000),
               csv_xlim=(0, 2.5), csv_ylim=(0, 30000))

In [ ]:
# Plot after-trimming alignment size, occupancy and conservation
plot_msa_stats(stats_trim, figsize=(28, 7),
               shape_xlim=(0, 4000), shape_ylim=(0, 700),
               occ_xlim=(0, 1), occ_ylim=(0, 14000),
               csv_xlim=(0, 2.5), csv_ylim=(0, 30000))

# SEED example

In [ ]:
# Plot MSA
def plot_msa(msa, label_size=18, title_size=24, figsize=(20, 10)):
    
    # Initialize new plot
    fig = plt.figure(constrained_layout=True, figsize=figsize)
    # # Add main title
    # fig.suptitle('Post-trimming multiple sequence alignment (MSA)')
    
    # Define axes grid
    grid = fig.add_gridspec(2, 3)

    # Initialize axis
    axs = [
        fig.add_subplot(grid[0, 0]),  # Axis for occupancy distribution
        fig.add_subplot(grid[0, 1:]),  # Axis for occupancy scatter
        fig.add_subplot(grid[1, 0]),  # Axis for conservation distribution
        fig.add_subplot(grid[1, 1:]),  # Axis for conservation scatter
    ]
    
    # Occupancy distribution
    _ = axs[0].set_title('Occupancy distribution', fontsize=title_size)
    _ = msa.plot_hist(score='occupancy', density=True, ax=axs[0])
    # Occupancy scatter
    _ = axs[1].set_title('Occupancy per aligned position', fontsize=title_size)
    _ = msa.plot_scatter(score='occupancy', ax=axs[1])

    # Conservation distribution
    _ = axs[2].set_title('Conservation distribution', fontsize=title_size)
    _ = msa.plot_hist(score='conservation', density=True, ax=axs[2])
    # Conservation scatter
    _ = axs[3].set_title('Conservation per aligned position', fontsize=title_size)
    _ = msa.plot_scatter(score='conservation', ax=axs[3])
    
    # Show plot
    plt.show()
    # Close plot
    plt.close()

In [ ]:
# Define single cluster path
CLUSTER_PATH = '/nfs/production/metagenomics/mgnifams/dclementel/MGnifam/tmp/mgseed2/MGYP001272227668'

# Define pre- and post- trimming SEED alignments paths
seed_pre_path = os.path.join(CLUSTER_PATH, 'SEED_raw')
seed_post_path = os.path.join(CLUSTER_PATH, 'SEED')

In [ ]:
# Before trimming SEED alignment
seed_pre = MSA.from_aln(seed_pre_path)
# Plot SEED alignment
plot_msa(seed_pre)

In [ ]:
# After trimming SEED alignment
seed_post = MSA.from_aln(seed_post_path)
# Plot SEED alignment
plot_msa(seed_post)

## Comparing computation time

In [ ]:
# Define x label
x = ['mgseed', 'pfbuild', 'mfbuild']
# Define y label
y_old = [3000, 0, 0]  # Computation times for old pipeline
y_new = [950, 0, 0]  # Computation times for new pipeline